# Before We Begin

## Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down
- Press "Save"

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

## Installing BERTopic

We start by installing BERTopic from PyPi:

In [2]:
%%capture
!pip install bertopic datasets

# **Tutorial** - Topic Modeling with BERTopic

In this tutorial we will be exploring how to use BERTopic to create topics from a dataset. The most frequent use-cases and methods are discussed together with important parameters to keep a look out for. 


## BERTopic
BERTopic is a topic modeling technique that leverages 🤗 transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions. 

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

## What is Topic Modelling and Why is it Useful

In machine learning (ML) and natural language processing (NLP), topic modeling is an unsupervised statistical method for discovering abstract categories that exist within a collection of documents.

# Data
For this example, we use [_A Real-Life Scenario Dialogue Summarization Dataset_](https://github.com/cylnlp/dialogsum)


In [3]:
from datasets import load_dataset

docs = load_dataset('knkarthick/dialogsum')['train']['summary']
f"Number of documents in the corpus {len(docs)}"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-8300315d3067c408/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

'Number of documents in the corpus 12460'

# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model. 




## Creating (Fitting) the Topic Model

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead. 

We will also calculate the topic probabilities to see how likely a topic to be present in a document. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model. 


In [6]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cuda")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True, calculate_probabilities=True)

# topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)

topics, probs = topic_model.fit_transform(docs)

Batches:   0%|          | 0/390 [00:00<?, ?it/s]

2022-09-08 09:22:58,019 - BERTopic - Transformed documents to Embeddings
2022-09-08 09:23:12,459 - BERTopic - Reduced dimensionality
2022-09-08 09:23:20,489 - BERTopic - Clustered reduced embeddings


## What Happened in the Previous Step
- Transformed documents to Embeddings
- Reduced dimensionality with [UMAP](https://umap-learn.readthedocs.io/en/latest/how_umap_works.html), which is relatively fast even on larger datasets with high (3+) dimensionality and similar points tend to cluster together in reduced dimensionality ([Video](https://youtu.be/eN0wFzBA4Sc))
- Clustered reduced embeddings using [HDBSCAN](https://hdbscan.readthedocs.io/en/latest/how_hdbscan_works.html)

**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents. 

In [10]:
topic_model.get_topic_info().head(20)

,Topic,Count,Name
0,-1,6783,-1_the_to_person1_and
1,0,271,0_birthday_party_dinner_invites
2,1,226,1_tomorrow_next_appointment_week
3,2,169,2_account_bank_open_money
4,3,161,3_suit_buys_dress_buy
5,4,138,4_job_company_work_experience
6,5,119,5_doctor_headache_dr_toothache
7,6,115,6_refuses_go_but_invites
8,7,104,7_china_beijing_chinese_shanghai
9,8,103,8_computer_internet_online_how


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [12]:
topic_model.get_topic(0)  # Select the most frequent topic

[('birthday', 0.03886603212825303),
 ('party', 0.033676966036410655),
 ('dinner', 0.024550416123890043),
 ('invites', 0.022376064376646996),
 ('christmas', 0.0149915578911493),
 ('saturday', 0.013291076133490962),
 ('celebrate', 0.012868730714439315),
 ('restaurant', 0.011995878237733426),
 ('friday', 0.010963227716426443),
 ('tomorrow', 0.010889530018244336)]

**NOTE**: BERTopic is stocastich which mmeans that the topics might differ across runs. This is mostly due to the stocastisch nature of UMAP.

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created. 

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good 
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation. 
Instead, we can visualize the topics that were generated in a way very similar to 
[LDAvis](https://github.com/cpsievert/LDAvis):

In [13]:
topic_model.visualize_topics()

## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can 
be used to understand how confident BERTopic is that certain topics can be found in a document. 

To visualize the distributions, we simply call:

In [15]:
topic_model.visualize_distribution(probs[0], min_probability=0.015)

## Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [17]:
topic_model.visualize_hierarchy(top_n_topics=25)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [20]:
topic_model.visualize_barchart(top_n_topics=10)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [24]:
topic_model.visualize_heatmap(n_clusters=100, width=1000, height=1000)

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created. 

This allows for fine-tuning the model to your specifications and wishes. 

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update 
the topic representation with new parameters for `c-TF-IDF`: 


In [33]:
topic_model.update_topics(docs, topics, n_gram_range=(1, 2))

In [34]:
topic_model.get_topic(1)   # We select topic that we viewed before

[('tomorrow', 0.017661912601782764),
 ('next', 0.01567888434264718),
 ('week', 0.011549162596024612),
 ('appointment', 0.011431623928503483),
 ('an appointment', 0.011136732745279486),
 ('morning', 0.0106934465794366),
 ('next week', 0.009415123864326115),
 ('will', 0.009011736587907236),
 ('mr', 0.008732740667702143),
 ('tuesday', 0.008431227005213234)]

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so, 
is that you can decide the number of topics after knowing how many are actually created. It is difficult to 
predict before training your model how many topics that are in your documents and how many will be extracted. 
Instead, we can decide afterwards how many topics seems realistic:





In [36]:
new_topics, new_probs = topic_model.reduce_topics(docs, topics, probs, nr_topics=60)

2022-09-08 10:14:49,215 - BERTopic - Reduced number of topics from 137 to 61


# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar 
to an input search_term. Here, we are going to be searching for topics that closely relate the 
search term "vehicle". Then, we extract the most similar topic and check the results: 

In [37]:
similar_topics, similarity = topic_model.find_topics("smell of onions", top_n=5); similar_topics

[9, 16, 45, 58, 36]

In [38]:
topic_model.get_topic(9)

[('pizza', 0.02917710137222477),
 ('food', 0.020055950540272125),
 ('restaurant', 0.017835769111722497),
 ('cook', 0.01781711910648788),
 ('but', 0.017233086967448484),
 ('person1', 0.01575891427815267),
 ('person2', 0.01566941374402496),
 ('the food', 0.014464005804480746),
 ('dinner', 0.013218012266730915),
 ('the', 0.01315702062840351)]

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved. 

In [ ]:
# Save model
topic_model.save("my_model")	

In [ ]:
# Load model
my_model = BERTopic.load("my_model")	